In [1]:
# imports
import pandas as pd
import requests
import csv
from datetime import datetime, timedelta

In [13]:
year = 2022
node = "SRP_PALOVERDE500-APND"

In [11]:
month_dict = {'Jan': ['0101', '0131'],
              'Feb': ['0201', '0228'],
              'Mar': ['0301', '0331'],
              'Apr': ['0401', '0430'],
              'May': ['0501', '0531'],
              'Jun': ['0601', '0630'],
              'Jul': ['0701', '0731'],
              'Aug': ['0801', '0831'],
              'Sep': ['0901', '0930'],
              'Oct': ['1001', '1031'],
              'Nov': ['1101', '1130'],
              'Dec': ['1201', '1231']
              }
dataframes = []
string1= 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime='
string2= 'T07:00-0000&enddatetime='
string3= 'T07:00-0000&market_run_id=DAM&node='


In [14]:
links = [f"{string1}{year}{month_dict[month][0]}{string2}{year}{month_dict[month][1]}{string3}{node}" for month in month_dict.keys()]

In [15]:
links

['http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220101T07:00-0000&enddatetime=20220131T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220201T07:00-0000&enddatetime=20220228T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220301T07:00-0000&enddatetime=20220331T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220401T07:00-0000&enddatetime=20220430T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220501T07:00-0000&enddatetime=20220531T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-AP

In [6]:
with open('links.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for link in links:
        writer.writerow([link])

In [7]:
links

['http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220101T07:00-0000&enddatetime=20220131T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220201T07:00-0000&enddatetime=20220228T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220301T07:00-0000&enddatetime=20220331T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220401T07:00-0000&enddatetime=20220430T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-APND',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220501T07:00-0000&enddatetime=20220531T07:00-0000&market_run_id=DAM&node=SRP_PALOVERDE500-AP

In [8]:
import csv
import requests
import zipfile
import io

# Define the list of links to ZIP files

# Initialize an empty list to store the data from the CSV files
data = []

# Loop through the ZIP links and retrieve the data from each CSV file
for link in links:
    response = requests.get(link)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    for file in zip_file.namelist():
        if file.endswith('.csv'):
            content = zip_file.read(file)
            decoded_content = content.decode('utf-8')
            csvreader = csv.reader(decoded_content.splitlines(), delimiter=',')
            for row in csvreader:
                data.append(row)

# Write the data to a new CSV file
with open('unfiltered.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for row in data:
        writer.writerow(row)

In [9]:
with open('unfiltered.csv', 'r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile)

    # Get the index of the "LMP_TYPE" column
    header = next(reader)
    lmp_type_index = header.index("LMP_TYPE")

    # Filter the rows to only include those where the value in the "LMP_TYPE" column is "LMP"
    filtered_rows = []
    for row in reader:
        if row[lmp_type_index] == "LMP":
            filtered_rows.append(row)

# Write the filtered rows to a new CSV file
with open('filtered.csv', 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)
    writer.writerows(filtered_rows)